In [ ]:
!pip install diffusers accelerate gradio
from diffusers import StableDiffusionPipeline
from IPython.display import clear_output
import torch
import gradio as gr
from itertools import product
import matplotlib.pyplot as plt


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
clear_output()

Вот тут посмотрел документацию:
https://huggingface.co/docs/diffusers/api/pipelines/stable_diffusion/overview \
Тут в качестве примера берется CompVis/stable-diffusion-v1-4

In [ ]:
model = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to(DEVICE)
clear_output()

In [23]:
'''Можно так попробовать и тут вводить промпты и настраивать num_inference_steps и guidance_scale'''
gr.Interface.from_pipeline(model).launch()

'''A stylized snake illustration, cartoonish style, bold lines, simple shapes, vibrant colors, playful design,
clear background, digital art, high resolution.
'''

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://188e578e5dd516a15f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Небольшое исследование: посмотрим на качество генерации в зависимости от num_inference_steps и guidance_scale

In [ ]:
def generate(prompt, num_inference_steps_list, guidance_scale_list):
  param_combinations = list(product(num_inference_steps_list, guidance_scale_list))
  fig, axes = plt.subplots(len(num_inference_steps_list), len(guidance_scale_list), figsize=(15, 15))

  for idx, (num_steps, guidance) in enumerate(param_combinations):
      row_idx = idx // len(guidance_scale_list)
      col_idx = idx % len(guidance_scale_list)
      ax = axes[row_idx, col_idx] if len(num_inference_steps_list) > 1 else axes[col_idx]

      # Генерация изображения
      generated_img = model(prompt=prompt, num_inference_steps=num_steps, guidance_scale=guidance).images[0]
      ax.imshow(generated_img)
      ax.set_title(f"num_inference_steps: {num_steps}, guidance_scale: {guidance}")
      ax.axis('off')


  plt.tight_layout()
  plt.show()

In [ ]:
PROMPT1 = '''A vibrant and colorful Christmas tree, lots of different shaped ornaments,
            bright lights, fun and joyful mood, cheerful atmosphere, festive decorations, high resolution.
          '''

PROMPT2 = '''A student sitting at his computer with a comical expression of irritation on his face, disheveled hair,
             eyes wide with embarrassment, an incomprehensible code on the screen, a crumpled paper ball on the table,
             a carefree mood, expressive cartoon style, high resolution, high detail.
          '''

NUM_INFERENCE_STEPS_LIST = [25, 50, 100]
GUIDANCE_SCALE_LIST = [5, 7.5, 10]

In [ ]:
generate(PROMPT1, NUM_INFERENCE_STEPS_LIST, GUIDANCE_SCALE_LIST)

Output hidden; open in https://colab.research.google.com to view.

In [22]:
generate(PROMPT2, NUM_INFERENCE_STEPS_LIST, GUIDANCE_SCALE_LIST)

Output hidden; open in https://colab.research.google.com to view.

**Комментарии** 
1. В чем суть DiT? \
 Основная суть DiT в том, чтобы использовать архитектуру трансформера при обратном диффузионном процессе (когда шум постепенно преобразуется в целевое изображение) в диффузионной модели. До этого обычно использовались сверточные нейронные сети, например U-Net. Такой подход позволяет лучше улавливать глобальные зависимости в данных, т.е. зависимости между разными частями изображения, что важно для понимания структуры и контекста изображения (за счет механизма внимания в трансформере), таким образом улучшается качество генерируемых объектов.

2. Какой подход используется для генерации? \
Подход здесь стандартный как и у других диффузионных моделей. Состоит из 2 этапов: \
а) Прямая диффузия - когда итеративно к скрытому представлению изображения (получается с помощью кодировщика VAE) добавляется небольшой гауссовский шум (до тех пор ,пока оно не превратится полностью в гауссовский шум) \
б) Обратная диффузия - Трансформер итеративно убирает шум из зашумленного изображения, т.е. восстанавливает исходное изображение. Он по сути обучается предсказывать шум, который добавлялся на каждой итерации при прямой диффузии, а этот уже предсказанный шум вычитается на каждом временном шаге. А потом уже полученное скрытое представление декодируется декодером VAE для получения уже финальной картинки, которой можно уже любоваться)

3. Как авторы модифицировали трансформер? Для чего? \
Из самого вопроса уже следует то, что исследователи сильно не меняли архитектуру трансформера - они лишь немного его модифицировали, адаптировали к решаемой задаче. Что изменилось: \
а) DiT не работает непосредственно с пикселями исходного изображения, он работает с закодированными с помощью VAE скрытыми представлениями картинок, которые уже дробятся на кусочки-патчи как в ViT-модели. Такой подход сильно снижает вычислительные затраты. \
б) Перед тем, как кусочки-патчи закодированного изображения попадают в трансформер, к ним добавляется информация о классе.  Трансформер "читает" эти ярлычки и понимает, какое изображение он должен сгенерировать. \
в)  Добавились "подсказки" о том, насколько шумным является изображение. Исследователи добавили к каждому патчу еще один ярлычок встраивания времени, который соответствует текущему шагу диффузии (насколько изображение зашумлено). Трансформер, получив этот ярлычок, знает, насколько сильно нужно "почистить" изображение от шума на текущем шаге.

Эти модификации были произведены для того, чтобы трансформер не только обрабатывал изображения, но и понимал контекст диффузии: какой тип изображения нужно сгенерировать и на каком этапе "очистки" изображения он находится. Это позволило делать более качественную и управляемую генерацию, а также были снижены вычислительные затраты.
